In [1]:
import os
import cv2
import numpy as np
from scipy.signal import fftconvolve
from skimage import img_as_float, io, color
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
from scipy.fft import fft2, ifft2, fftshift
from typing import Tuple

# -------------------------
# Utility functions
# -------------------------
def load_images_from_folder(folder: str, ext=('png','jpg','jpeg','bmp')) -> list:
    imgs = []
    files = sorted([os.path.join(folder,f) for f in os.listdir(folder) if f.lower().endswith(tuple(ext))])
    for f in files:
        im = img_as_float(io.imread(f))
        if im.ndim == 2:  # grayscale -> convert to 3-channel for uniformity
            im = np.stack([im]*3, axis=-1)
        elif im.shape[2] == 4:
            im = im[..., :3]  # drop alpha
        imgs.append((os.path.basename(f), im))
    return imgs

def save_img(path: str, img: np.ndarray):
    img = np.clip(img*255, 0, 255).astype(np.uint8)
    cv2.imwrite(path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

# -------------------------
# Degradation: blur + gaussian noise
# -------------------------
def gaussian_kernel(k_size: int, sigma: float) -> np.ndarray:
    ax = np.arange(-k_size//2 + 1., k_size//2 + 1.)
    xx, yy = np.meshgrid(ax, ax)
    kernel = np.exp(-(xx**2 + yy**2) / (2. * sigma**2))
    kernel = kernel / np.sum(kernel)
    return kernel

def degrade_image(img: np.ndarray, kernel: np.ndarray, sigma_noise: float) -> np.ndarray:
    # Blur each channel with same kernel via FFT-based convolution for speed
    out = np.zeros_like(img)
    for c in range(img.shape[2]):
        out[..., c] = fftconvolve(img[..., c], kernel, mode='same')
    # Add Gaussian noise (zero mean)
    noisy = out + np.random.normal(0, sigma_noise, out.shape)
    return np.clip(noisy, 0, 1)

# -------------------------
# Frequency helpers
# -------------------------
def pad_psf(psf: np.ndarray, shape: Tuple[int,int]) -> np.ndarray:
    psf_padded = np.zeros(shape, dtype=np.float32)
    kh, kw = psf.shape
    psf_padded[:kh, :kw] = psf
    # circularly shift so kernel center is at (0,0) freq origin
    psf_padded = np.roll(psf_padded, -kh//2, axis=0)
    psf_padded = np.roll(psf_padded, -kw//2, axis=1)
    return psf_padded


In [2]:
pip install opencv-python


   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   -- ------------------------------------- 2.6/39.0 MB 15.1 MB/s eta 0:00:03
   ----- ---------------------------------- 5.2/39.0 MB 14.0 MB/s eta 0:00:03
   ------- -------------------------------- 7.6/39.0 MB 12.9 MB/s eta 0:00:03
   ---------- ----------------------------- 10.5/39.0 MB 13.3 MB/s eta 0:00:03
   ------------- -------------------------- 13.4/39.0 MB 13.2 MB/s eta 0:00:02
   ---------------- ----------------------- 16.0/39.0 MB 13.2 MB/s eta 0:00:02
   ------------------- -------------------- 18.9/39.0 MB 13.3 MB/s eta 0:00:02
   ---------------------- ----------------- 21.5/39.0 MB 13.3 MB/s eta 0:00:02
   ------------------------- -------------- 24.4/39.0 MB 13.3 MB/s eta 0:00:02
   --------------------------- ------------ 27.3/39.0 MB 13.3 MB/s eta 0:00:01
   ------------------------------ --------- 29.6/39.0 MB 13.0 MB/s eta 0:00:01
   --------------------------------- ------ 32.8/39.0 MB 13.2 MB


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install opencv-python scipy scikit-image numpy


  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   -------- ------------------------------- 2.9/12.9 MB 14.9 MB/s eta 0:00:01
   ----------------- ---------------------- 5.8/12.9 MB 14.0 MB/s eta 0:00:01
   -------------------------- ------------- 8.4/12.9 MB 13.8 MB/s eta 0:00:01
   ---------------------------------- ----- 11.3/12.9 MB 13.7 MB/s eta 0:00:01
   ---------------------------------------- 12.9/12.9 MB 12.9 MB/s eta 0:00:00
Using cached networkx-3.5-py3-none-any.whl (2.0 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# -------------------------
# Inverse filter (frequency domain)
# -------------------------
def inverse_filter(degraded: np.ndarray, psf: np.ndarray, eps: float=1e-3) -> np.ndarray:
    h, w, c = degraded.shape
    H = fft2(pad_psf(psf, (h, w)))
    restored = np.zeros_like(degraded)
    # Prevent division by zero: add small eps
    denom = H.copy()
    denom[np.abs(denom) < eps] = eps
    for ch in range(c):
        G = fft2(degraded[..., ch])
        Fhat = G / denom
        fhat = np.real(ifft2(Fhat))
        restored[..., ch] = fhat
    restored = np.clip(restored, 0, 1)
    return restored

# -------------------------
# Wiener filter (frequency domain)
# Implemented as: F = (H* / (|H|^2 + K)) * G
# where K = S_n / S_f  (noise-to-signal power ratio). We approximate K as a scalar.
# -------------------------
def wiener_filter(degraded: np.ndarray, psf: np.ndarray, K: float=0.01) -> np.ndarray:
    h, w, c = degraded.shape
    H = fft2(pad_psf(psf, (h, w)))
    H_conj = np.conj(H)
    H_abs2 = np.abs(H)**2
    restored = np.zeros_like(degraded)
    denom = H_abs2 + K
    for ch in range(c):
        G = fft2(degraded[..., ch])
        Fhat = (H_conj / denom) * G
        fhat = np.real(ifft2(Fhat))
        restored[..., ch] = fhat
    restored = np.clip(restored, 0, 1)
    return restored

# -------------------------
# Evaluation metrics (PSNR and SSIM per-channel averaged)
# -------------------------
def evaluate(restored: np.ndarray, original: np.ndarray) -> Tuple[float, float]:
    # Convert to grayscale for SSIM (recommended), but PSNR computed on color
    psnr_val = psnr(original, restored, data_range=1.0)
    # compute SSIM on luminance (convert to grayscale)
    orig_gray = color.rgb2gray(original)
    rest_gray = color.rgb2gray(restored)
    ssim_val = ssim(orig_gray, rest_gray, data_range=1.0)
    return psnr_val, ssim_val


In [6]:
# -------------------------
# Main pipeline
# -------------------------
def process_folder(input_folder: str, output_folder: str,
                   kernel_size: int=15, kernel_sigma: float=3.0,
                   noise_sigma: float=0.02, K_est: float=0.01):
    os.makedirs(output_folder, exist_ok=True)
    imgs = load_images_from_folder(input_folder)
    if not imgs:
        print("No images found in folder:", input_folder)
        return

    psf = gaussian_kernel(kernel_size, kernel_sigma)

    summary = []
    for name, img in imgs:
        print("Processing:", name)
        degraded = degrade_image(img, psf, noise_sigma)
        inv_rest = inverse_filter(degraded, psf, eps=1e-3)
        wiener_rest = wiener_filter(degraded, psf, K=K_est)

        # Evaluate
        psnr_degraded, ssim_degraded = evaluate(degraded, img)
        psnr_inv, ssim_inv = evaluate(inv_rest, img)
        psnr_wiener, ssim_wiener = evaluate(wiener_rest, img)

        print(f"  Degraded PSNR: {psnr_degraded:.2f}, SSIM: {ssim_degraded:.3f}")
        print(f"  Inverse PSNR:  {psnr_inv:.2f}, SSIM: {ssim_inv:.3f}")
        print(f"  Wiener PSNR:   {psnr_wiener:.2f}, SSIM: {ssim_wiener:.3f}")

        base = os.path.splitext(name)[0]
        save_img(os.path.join(output_folder, base + "_orig.png"), img)
        save_img(os.path.join(output_folder, base + "_degraded.png"), degraded)
        save_img(os.path.join(output_folder, base + "_inv.png"), inv_rest)
        save_img(os.path.join(output_folder, base + "_wiener.png"), wiener_rest)

        summary.append({
            'name': name,
            'psnr_degraded': psnr_degraded,
            'ssim_degraded': ssim_degraded,
            'psnr_inverse': psnr_inv,
            'ssim_inverse': ssim_inv,
            'psnr_wiener': psnr_wiener,
            'ssim_wiener': ssim_wiener
        })

    # Print summary
    print("\nSummary (averages):")
    avg = {k: np.mean([s[k] for s in summary]) for k in summary[0].keys() if k != 'name'}
    for k,v in avg.items():
        print(f"  {k}: {v:.4f}")

if __name__ == "__main__":
    # Example usage:
    # Put dataset images (e.g., BSDS500 images) in ./data/images/
    # Outputs will be saved in ./results/
    input_folder = "C:\\Users\\soham\\OneDrive\\Desktop\\fdip Assignments\\natural_images\\airplane"   # <-- change to your images folder
    output_folder = "./results"
    process_folder(input_folder, output_folder,
                   kernel_size=15, kernel_sigma=3.0,
                   noise_sigma=0.02, K_est=0.01)


Processing: airplane_0000.jpg
  Degraded PSNR: 19.72, SSIM: 0.542
  Inverse PSNR:  5.00, SSIM: 0.006
  Wiener PSNR:   16.45, SSIM: 0.551
Processing: airplane_0001.jpg
  Degraded PSNR: 18.46, SSIM: 0.602
  Inverse PSNR:  4.34, SSIM: 0.005
  Wiener PSNR:   15.64, SSIM: 0.577
Processing: airplane_0002.jpg
  Degraded PSNR: 20.28, SSIM: 0.485
  Inverse PSNR:  4.97, SSIM: 0.007
  Wiener PSNR:   18.44, SSIM: 0.506
Processing: airplane_0003.jpg
  Degraded PSNR: 19.09, SSIM: 0.575
  Inverse PSNR:  4.62, SSIM: 0.007
  Wiener PSNR:   16.70, SSIM: 0.557
Processing: airplane_0004.jpg
  Degraded PSNR: 18.60, SSIM: 0.557
  Inverse PSNR:  4.52, SSIM: 0.008
  Wiener PSNR:   15.77, SSIM: 0.554
Processing: airplane_0005.jpg
  Degraded PSNR: 18.36, SSIM: 0.423
  Inverse PSNR:  5.10, SSIM: 0.009
  Wiener PSNR:   17.88, SSIM: 0.519
Processing: airplane_0006.jpg
  Degraded PSNR: 18.63, SSIM: 0.585
  Inverse PSNR:  4.73, SSIM: 0.004
  Wiener PSNR:   15.92, SSIM: 0.544
Processing: airplane_0007.jpg
  Degraded 